Problem statement

An e-commerce platform is getting all geared up for a season clearance sale and plans to leverage social media as the primary channel to reach their audiences. The campaign’s target group are individuals/families that have recently posted a picture of their indoor Christmas decor or are traveling during the holidays. 

You work at a leading social media platform and your team has been tasked to build a product recommendation engine for consumers. As part of the development team, your role is to use Deep Learning to develop a model that classifies images based on elements within the picture. These elements should be related to decor or holiday season vacations, such as a snowman, a Christmas tree, flights, and the like. 

# In this notebook I will be using transfer learning and data augmentation for model training and predicting the result.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Reading data

In [ ]:
import pandas as pd

In [ ]:
train_df=pd.read_csv('../input/hackereath-holiday-season-deep-learning-contest/dataset/train.csv')

In [ ]:
train_df.head()

In [ ]:
train_df['Class'].value_counts()

In [ ]:
from keras.preprocessing import image

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
img=image.load_img('../input/hackereath-holiday-season-deep-learning-contest/dataset/train/image1000.jpg')

In [ ]:
img=image.img_to_array(img)/255

In [ ]:
plt.imshow(img)

In [ ]:
print(img.shape)

# Dividing data in valid and train part using StratifiedKFold as data is skewed

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
train_df['Kfold']=-1

In [ ]:
train_df.head()

In [ ]:
train_df=train_df.sample(frac=1).reset_index(drop=True)

In [ ]:
train_df.tail()

In [ ]:
y=train_df['Class']

In [ ]:
kf=StratifiedKFold(n_splits=5)

In [ ]:
for f,(t_,v_) in enumerate(kf.split(X=train_df,y=y)):
    train_df.loc[v_,'Kfold']=f

In [ ]:
train_df.head()

In [ ]:
train=train_df[train_df['Kfold']!=4]

In [ ]:
valid=train_df[train_df['Kfold']==4]

In [ ]:
valid.tail()

In [ ]:
valid['Class'].value_counts()

# As the data is large so it will be better to use datagenerator, so I am using keras Imagedatagenerator and we can do data augmentation in this step

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         width_shift_range=0.1,
#         height_shift_range=0.1)

train_generator=train_datagen.flow_from_dataframe(dataframe=train,
                                            directory="../input/hackereath-holiday-season-deep-learning-contest/dataset/train/",
                                            x_col="Image",
                                            y_col="Class",
                                            subset="training",
                                            batch_size=128,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="categorical",
                                            target_size=(331,331))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

valid_datagen = ImageDataGenerator(rescale=1./255)
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         width_shift_range=0.1,
#         height_shift_range=0.1)

valid_generator=valid_datagen.flow_from_dataframe(dataframe=valid,
                                            directory="../input/hackereath-holiday-season-deep-learning-contest/dataset/train/",
                                            x_col="Image",
                                            y_col="Class",
                                            subset="training",
                                            batch_size=128,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="categorical",
                                            target_size=(331,331))

# Loading nasnet large model and setting all layers except last 35 as non trainable so we can generalise our model on our data and also do transfer learning for better results

In [ ]:
from keras.applications.nasnet import NASNetLarge
# from keras.applications.resnet50 import preprocess_input,decode_predictions
from keras.layers import Input, Conv2D, MaxPool2D, Dense, Flatten
from keras.models import Model
from keras.utils import to_categorical

In [ ]:
resnet=NASNetLarge(include_top=True,weights='imagenet')

In [ ]:
x=resnet.layers[-2].output
fc1=Dense(6,activation='softmax')(x)

In [ ]:
my_model=Model(inputs=resnet.input,outputs=fc1)

In [ ]:
# my_model.summary()

In [ ]:
from keras.optimizers import Adam

In [ ]:
adam=Adam(learning_rate=0.0001)

In [ ]:
for l in my_model.layers[:-5]:
    #print(l)
    l.trainable = False
my_model.compile(optimizer='adam',loss ="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
my_model.fit_generator(train_generator,steps_per_epoch=5176//128,validation_data=valid_generator,validation_steps=1293//128,epochs=2)

In [ ]:
my_model.save('model.h5')

In [ ]:
# import keras

In [ ]:
# my_model=keras.models.load_model('./model.h5')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    directory='../input/hackereath-holiday-season-deep-learning-contest/dataset/test/',
    target_size=(331, 331),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

# above test datagenerator code is not working, reason is that we must have another folder inside the test image folder that contains all test images for datagenerator(read_from_directory), So let's do this

In [ ]:
import shutil

In [ ]:
import os

In [ ]:
os.mkdir('test_data')

In [ ]:
os.mkdir('test_data/test')

In [ ]:
for file in os.listdir('../input/hackereath-holiday-season-deep-learning-contest/dataset/test'):
    shutil.copyfile('../input/hackereath-holiday-season-deep-learning-contest/dataset/test/' + file,'./test_data/test/' + file)

In [ ]:
len(os.listdir('./test_data/test'))

In [ ]:
test_generator = test_datagen.flow_from_directory(
    directory='./test_data/',
    target_size=(331, 331),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

In [ ]:
y_pred=my_model.predict_generator(test_generator,
verbose=1)

In [ ]:
y_pred_2=np.argmax(y_pred,axis=1)

In [ ]:
train_datagen.class_indices

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in y_pred_2]

In [ ]:
predictions[:10]

In [ ]:
test_name=os.listdir('./test_data/test')

# But I am not using above method you can try this

In [ ]:
import os

In [ ]:
name=[]
y_pred=[]

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

In [ ]:
import numpy as np

In [ ]:
s=0
for i in os.listdir('../input/hackereath-holiday-season-deep-learning-contest/dataset/test/'):
    name.append(i)
    i='../input/hackereath-holiday-season-deep-learning-contest/dataset/test/'+i
    img=image.load_img(i,target_size=(331,331,3))
    img=image.img_to_array(img)/255
    pred=my_model.predict(img.reshape(1,331,331,3))
    y_pred.append(labels[np.argmax(pred[0])])
    s+=1
    if s%100==0:
        print(s)

    

In [ ]:
data=pd.DataFrame((zip(name,y_pred)),columns=['Image','Class'])

In [ ]:
data.head()

In [ ]:
data.to_csv('submission.csv',index=False)


In [ ]:
data.shape

# Done.
## Now there are many ways to improve the results,like changing the trainable layers and model and other try and make good results.
## Please Upvote the notebook if it was helpful